<a href="https://colab.research.google.com/github/ogut77/DataScience/blob/main/BoostingClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# for Jupyter-book, we copy data from GitHub, locally, to save Internet traffic,
# you can specify the data/ folder from the root of your cloned
# https://github.com/Yorko/mlcourse.ai repo, to save Internet traffic
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
from matplotlib import pyplot as plt
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/"
df = pd.read_csv(DATA_PATH + "telecom_churn.csv")

df["International plan"] = pd.factorize(df["International plan"])[0]
df["Voice mail plan"] = pd.factorize(df["Voice mail plan"])[0]
df["Churn"] = df["Churn"].astype("int")
states = df["State"]
y = df["Churn"]
X=df.drop(["State", "Churn"], axis=1)


In [2]:
X

,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls
0,128,415,0,0,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1
1,107,415,0,0,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1
2,137,415,0,1,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0
3,84,408,1,1,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2
4,75,415,1,1,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3328,192,415,0,0,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2
3329,68,415,0,1,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3
3330,28,510,0,1,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2
3331,184,510,1,1,0,213.8,105,36.35,159.6,84,13.57,139.2,137,6.26,5.0,10,1.35,2


In [12]:
y

0       0
1       0
2       0
3       0
4       0
       ..
3328    0
3329    0
3330    0
3331    0
3332    0
Name: Churn, Length: 3333, dtype: int64

In [4]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=17
)



In [7]:
#Decison Tree
from sklearn.metrics import accuracy_score,confusion_matrix
tree = DecisionTreeClassifier(random_state=17)
tree.fit(X_train, y_train)

# make predictions for test data
tree_pred = tree.predict(X_test)
accuracy_score(y_test, tree_pred) 

0.92

In [10]:
print(confusion_matrix(y_test, tree_pred)) 
sum(y_test)

[[827  40]
 [ 40  93]]


133

In [19]:
#Random Forest 
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(random_state=17 )
forest.fit(X_train, y_train)

# make predictions for test data

forest_pred = forest.predict(X_test)
accuracy_score(y_test, forest_pred) 

0.95

In [15]:
# XGBoost
from xgboost import XGBClassifier
model = XGBClassifier(random_state=17)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.951

In [20]:
#Light GBM
import lightgbm as lgb
lgb_model = lgb.LGBMClassifier(random_state=17)
lgb_model.fit(X_train, y_train)
# make predictions for test data
accuracy_score(y_test, lgb_model.predict(X_test))

0.956

In [13]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier
gbm_model = GradientBoostingClassifier(random_state=17)
gbm_model.fit(X_train, y_train)
accuracy_score(y_test, gbm_model.predict(X_test))

0.954

In [22]:
#Ada Boost
from sklearn.ensemble import AdaBoostClassifier
ada = AdaBoostClassifier(random_state=17)
ada.fit(X_train, y_train)
accuracy_score(y_test, ada.predict(X_test))

0.874

In [26]:
#Random Forest CV
from time import time
t0=time()
from sklearn.model_selection import GridSearchCV, cross_val_score
forest_params = {"max_depth": range(5, X_train.shape[1],3), "max_features": range(5, X_train.shape[1],3)}
forest = RandomForestClassifier(random_state=17 )
forest_grid = GridSearchCV(forest, forest_params, cv=3, n_jobs=-1, verbose=True)
forest_grid.fit(X_train, y_train)
forest_grid.best_params_, forest_grid.best_score_
print(accuracy_score(y_test, forest_grid.predict(X_test)))
print(forest_grid.best_params_, forest_grid.best_score_)
t1=time()
print('Time is ' +str(t1-t0))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
0.954
{'max_depth': 17, 'max_features': 8} 0.946421265187332
Time is 51.27836489677429


In [25]:
 X_train.shape[1]

18

In [ ]:
#XGBoost CV

In [27]:
#Light GBM CV
import lightgbm as lgb


from time import time
t0=time()
from sklearn.model_selection import GridSearchCV, cross_val_score
lgb_params = {"max_depth": range(5, X_train.shape[1],3), "max_features": range(5, X_train.shape[1],3)}
lgbm = lgb.LGBMClassifier(random_state=17)
lgb_grid = GridSearchCV(lgbm, lgb_params , cv=3, n_jobs=-1, verbose=True)
lgb_grid.fit(X_train, y_train)
lgb_grid.best_params_, lgb_grid.best_score_
print(accuracy_score(y_test, lgb_grid.predict(X_test)))
print(lgb_grid.best_params_, lgb_grid.best_score_)
t1=time()
print('Time is ' +str(t1-t0))

Fitting 3 folds for each of 25 candidates, totalling 75 fits
0.954
{'max_depth': 17, 'max_features': 5} 0.9558526576521436
Time is 12.120943307876587


In [ ]:
#Gradient Based CV

In [28]:
!pip install optuna


     |████████████████████████████████| 308 kB 2.9 MB/s 
     |████████████████████████████████| 81 kB 7.4 MB/s 
     |████████████████████████████████| 210 kB 58.2 MB/s 
     |████████████████████████████████| 78 kB 6.4 MB/s 
     |████████████████████████████████| 113 kB 63.3 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 
     |████████████████████████████████| 150 kB 58.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=3148a7c87c104a5337164a6abc4df974e7e140e2fe9dced81e6ac0b062b0c0c9
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_float('x', -10, 10)
    return (x - 2) ** 2

study = optuna.create_study()
study.optimize(objective, n_trials=100)

print(study.best_params) # E.g. {'x': 2.002108042}

In [30]:
 print(study.best_trial)

FrozenTrial(number=62, values=[1.6179161434112966e-06], datetime_start=datetime.datetime(2022, 3, 11, 12, 34, 9, 195836), datetime_complete=datetime.datetime(2022, 3, 11, 12, 34, 9, 199282), params={'x': 1.998728026673467}, distributions={'x': UniformDistribution(high=10.0, low=-10.0)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=62, state=TrialState.COMPLETE, value=None)


In [ ]:
import optuna

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    x, y = X_train,y_train

    classifier_name = trial.suggest_categorical("classifier", ["GradientBoostingClassifier", "LightGBM"])
    if classifier_name == "GradientBoostingClassifier":
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         classifier_obj = sklearn.ensemble.GradientBoostingClassifier(random_state=17,  max_depth=max_depth, max_features=max_features )
        
    else:
         import lightgbm as lgb
         max_depth = trial.suggest_int("max_depth", 2,16)
         max_features = trial.suggest_int("max_features", 2,X_train.shape[1])
         classifier_obj = lgb.LGBMClassifier(random_state=17,  max_depth=max_depth, max_features=max_features )
        

    accuracy=sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3).mean()
   
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)
    print(study.best_trial)

In [32]:
#Print Best Model
study.best_params

{'classifier': 'LightGBM', 'max_depth': 12, 'max_features': 2}

In [48]:
#Apply Best Model Parameter to Test Data
import optuna

import sklearn.datasets
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm


# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    x, y = X_train,y_train

    param = {
        "objective": "binary",
        "metric": "binary_logloss",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    import lightgbm as lgb
    classifier_obj = lgb.LGBMClassifier(random_state=17,**param)

    accuracy=sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3).mean()
   
    return accuracy


if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=200)
    print(study.best_trial)

[I 2022-03-12 23:39:49,921] A new study created in memory with name: no-name-2facc234-be6e-4e17-bc3c-fc7793c7b1e2
[I 2022-03-12 23:39:52,645] Trial 0 finished with value: 0.9095603131592848 and parameters: {'lambda_l1': 1.2068594818664454e-05, 'lambda_l2': 3.103103899084126e-05, 'num_leaves': 46, 'feature_fraction': 0.4473570931046189, 'bagging_fraction': 0.5096144022717055, 'bagging_freq': 3, 'min_child_samples': 34}. Best is trial 0 with value: 0.9095603131592848.
[I 2022-03-12 23:39:53,297] Trial 1 finished with value: 0.9087034151301504 and parameters: {'lambda_l1': 0.0019840259744407637, 'lambda_l2': 0.04149308592878834, 'num_leaves': 79, 'feature_fraction': 0.4436929363387725, 'bagging_fraction': 0.4387269261479715, 'bagging_freq': 7, 'min_child_samples': 9}. Best is trial 0 with value: 0.9095603131592848.
[I 2022-03-12 23:39:54,208] Trial 2 finished with value: 0.9502767549040043 and parameters: {'lambda_l1': 2.9028261017708118e-08, 'lambda_l2': 0.00016044919551036927, 'num_leav

FrozenTrial(number=195, values=[0.9571385561102784], datetime_start=datetime.datetime(2022, 3, 12, 23, 42, 1, 474418), datetime_complete=datetime.datetime(2022, 3, 12, 23, 42, 2, 220217), params={'lambda_l1': 1.8685159677967946e-07, 'lambda_l2': 0.00018007857331122225, 'num_leaves': 163, 'feature_fraction': 0.9538555368068244, 'bagging_fraction': 0.9861918681410792, 'bagging_freq': 6, 'min_child_samples': 20}, distributions={'lambda_l1': LogUniformDistribution(high=10.0, low=1e-08), 'lambda_l2': LogUniformDistribution(high=10.0, low=1e-08), 'num_leaves': IntUniformDistribution(high=256, low=2, step=1), 'feature_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_fraction': UniformDistribution(high=1.0, low=0.4), 'bagging_freq': IntUniformDistribution(high=7, low=1, step=1), 'min_child_samples': IntUniformDistribution(high=100, low=5, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=195, state=TrialState.COMPLETE, value=None)


In [50]:
 print(study.best_params)

{'lambda_l1': 1.8685159677967946e-07, 'lambda_l2': 0.00018007857331122225, 'num_leaves': 163, 'feature_fraction': 0.9538555368068244, 'bagging_fraction': 0.9861918681410792, 'bagging_freq': 6, 'min_child_samples': 20}


In [56]:
param={'lambda_l1': 1.8685159677967946e-07, 'lambda_l2': 0.00018007857331122225, 'num_leaves': 163, 'feature_fraction': 0.9538555368068244, 'bagging_fraction': 0.9861918681410792, 'bagging_freq': 6, 'min_child_samples': 20}

In [55]:
import lightgbm as lgb
model = lgb.LGBMClassifier(random_state=17,**param)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.953

In [40]:

def objective(trial):
        x, y = X_train,y_train
        from xgboost import XGBClassifier
        random_state = 17
        n_jobs =  -1
        verbosity = 0
        objective = "binary:logistic"
        booster = trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"])
        alpha = trial.suggest_float("alpha", 1e-8, 1.0, log=True)
        max_depth= trial.suggest_int("max_depth", 2,16)  
        max_features= trial.suggest_int("max_features", 2,X_train.shape[1])
        eta = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        gamma = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        grow_policy = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
        subsample = trial.suggest_float("subsample", 0.1, 1.0, log = True)
        min_child_weight = trial.suggest_int("min_child_weight", 1, 15)
        colsample_bytree = trial.suggest_float("subsample", 0.1, 1.0, log=True)
        n_estimators = trial.suggest_int("n_estimators", 80, 120)
    

        classifier_obj = XGBClassifier( random_state= random_state,
                                            n_jobs          = n_jobs,
                                            verbosity        = verbosity, 
                                            objective        = objective,
                                            booster          = booster,
                                            alpha            = alpha,
                                            max_depth        =max_depth,
                                            max_features     =max_features,
                                            eta              = eta,
                                            gamma            = gamma,
                                            grow_policy      = grow_policy,
                                            subsample        = subsample,
                                            min_child_weight = min_child_weight,
                                            colsample_bytree = colsample_bytree,
                                            n_estimators     = n_estimators
                                            )
        

        accuracy=sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3).mean()
   
        return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 100)

trial = study.best_trial


print("Best hyperparameters: {}".format(trial.params))

[I 2022-03-12 23:30:45,105] A new study created in memory with name: no-name-33d41b86-73c7-4275-a4ef-454519158279
[I 2022-03-12 23:30:45,747] Trial 0 finished with value: 0.8499788808272098 and parameters: {'booster': 'gblinear', 'alpha': 0.0058598028672546725, 'max_depth': 3, 'max_features': 8, 'eta': 1.6661138130065074e-06, 'gamma': 0.00014394675614796864, 'grow_policy': 'lossguide', 'subsample': 0.7557670565878365, 'min_child_weight': 11, 'n_estimators': 113}. Best is trial 0 with value: 0.8499788808272098.
[I 2022-03-12 23:30:46,142] Trial 1 finished with value: 0.8979861242072039 and parameters: {'booster': 'gbtree', 'alpha': 0.0001418703394466687, 'max_depth': 12, 'max_features': 7, 'eta': 0.19063453599895724, 'gamma': 1.3733929297704288e-07, 'grow_policy': 'depthwise', 'subsample': 0.3013350446195821, 'min_child_weight': 3, 'n_estimators': 105}. Best is trial 1 with value: 0.8979861242072039.
[I 2022-03-12 23:30:46,489] Trial 2 finished with value: 0.8949880839340993 and paramet

Best hyperparameters: {'booster': 'dart', 'alpha': 0.2553983989210158, 'max_depth': 13, 'max_features': 16, 'eta': 0.00020746845752260047, 'gamma': 8.330886251031127e-05, 'grow_policy': 'lossguide', 'subsample': 0.9909853737540923, 'min_child_weight': 1, 'n_estimators': 95}


In [42]:
params={'booster': 'dart', 'alpha': 0.2553983989210158, 'max_depth': 13, 'max_features': 16, 'eta': 0.00020746845752260047, 'gamma': 8.330886251031127e-05, 'grow_policy': 'lossguide', 'subsample': 0.9909853737540923, 'min_child_weight': 1, 'n_estimators': 95}

In [47]:
from xgboost import XGBClassifier
model = XGBClassifier(random_state=17,**params)
model.fit(X_train, y_train)
# make predictions for test data
xgb_pred = model.predict(X_test)
accuracy_score(y_test, xgb_pred)

0.951